In [1]:
import argparse
import numpy as np
import sys, os
import torch
import time
import h5py
import torch.nn as nn
from sklearn import decomposition

#import custom modules
code_dir = '/user_data/mmhender/imStat/code/'
sys.path.append(code_dir)
from utils import prf_utils, torch_utils, texture_utils, default_paths
from model_fitting import initialize_fitting
from feature_extraction import texture_statistics_pyramid2 as texture_statistics_pyramid

device = initialize_fitting.init_cuda()


#device: 1
device#: 0
device name: GeForce GTX TITAN X

torch: 1.8.1+cu111
cuda:  11.1
cudnn: 8005
dtype: torch.float32


In [2]:
from utils import nsd_utils
batch_size=2
subject=1
n_sf = 4; 
n_ori = 4;
debug=True
use_node_storage = False


In [3]:
fn2save = os.path.join(pyramid_texture_feat_path, 'S%d_features_each_prf_%dori_%dsf.h5py'%(subject, n_ori, n_sf))
 

NameError: name 'pyramid_texture_feat_path' is not defined

In [ ]:
image_inds = np.arange(0,10)
print('Loading pre-computed features from %s'%fn2save)
t = time.time()
with h5py.File(fn2save, 'r') as data_set:
    values = np.copy(data_set['/features'][image_inds,:,:])
    data_set.close() 
elapsed = time.time() - t
print('Took %.5f seconds to load file'%elapsed)

# self.features_each_prf = values[image_inds,:,:]

In [22]:
values[1,0:10,0]

array([ 4.54796582e-01,  4.46013734e-03, -2.52191448e+00,  1.96903534e+01,
        3.85581143e-03,  9.86981571e-01,  1.44901156e+00,  3.12045217e+00,
        5.94274855e+00,  6.62372065e+00])

In [23]:
features_each_prf[1,0:10,0]

array([ 4.54796582e-01,  4.46013734e-03, -2.52191448e+00,  1.96903534e+01,
        3.85581143e-03,  9.86981571e-01,  1.44901156e+00,  3.12045217e+00,
        5.94274855e+00,  6.62372065e+00])

In [4]:
if use_node_storage:
    pyramid_texture_feat_path = default_paths.pyramid_texture_feat_path_localnode
else:
    pyramid_texture_feat_path = default_paths.pyramid_texture_feat_path

In [5]:

# Load and prepare the image set to work with (all images for the current subject, 10,000 ims)
stim_root = default_paths.stim_root
image_data = nsd_utils.get_image_data(subject)  
image_data = nsd_utils.image_uncolorize_fn(image_data)


Loading images for subject 1

image data size: (10000, 3, 227, 227) , dtype: uint8 , value range: 0 255


In [6]:
# Need to have size a multiple of 8, for the pyramid to work right
process_at_size=240
resample_fn = torch.nn.Upsample((process_at_size, process_at_size), mode="bilinear", align_corners=True)

# Params for the spatial aspect of the model (possible pRFs)
aperture = 1.0
aperture_rf_range = 1.1
aperture, models = initialize_fitting.get_prf_models(aperture_rf_range=aperture_rf_range)    

# Set up the pyramid
batch_size=2
feature_types_exclude = []
n_prf_sd_out = 2
do_varpart=False # this doesn't do anything here
group_all_hl_feats = False # this doesn't do anything here


most extreme RF positions:
[-0.55 -0.55  0.04]
[0.55       0.55       0.40000001]


In [9]:
np.cumsum(_feature_extractor.feature_type_dims_all)

array([  6,  22,  38,  48,  49, 321, 394, 419, 443, 467, 515, 611, 621,
       641])

In [15]:
_feature_extractor.feature_column_labels[37]

2

In [7]:
compute_features = True
_fmaps_fn = texture_statistics_pyramid.steerable_pyramid_extractor(pyr_height = n_sf, n_ori = n_ori)
# Initialize the "texture" model which builds on first level feature maps
_feature_extractor = texture_statistics_pyramid.texture_feature_extractor(_fmaps_fn,sample_batch_size=batch_size, feature_types_exclude=feature_types_exclude, \
                                               n_prf_sd_out=n_prf_sd_out, aperture=aperture, do_varpart = do_varpart, \
                              compute_features=compute_features, group_all_hl_feats = group_all_hl_feats, device=device)

n_features = _feature_extractor.n_features_total
n_images = image_data.shape[0]
n_prfs = models.shape[0]
n_batches = int(np.ceil(n_images/batch_size))

features_each_prf = np.zeros((n_images, n_features, n_prfs))

bb=0

# for bb in range(n_batches):

#     if debug and bb>1:
#         continue

batch_inds = np.arange(batch_size * bb, np.min([batch_size * (bb+1), n_images]))

print('Extracting features for images [%d - %d]'%(batch_inds[0], batch_inds[-1]))

image_batch = torch_utils._to_torch(image_data[batch_inds,:,:,:], device)

image_batch = resample_fn(image_batch)

_feature_extractor.clear_big_features()

mm=204

#     for mm in range(n_prfs):

#         if debug and mm>1:
#             continue

x,y,sigma = models[mm,:]
print('Getting features for pRF [x,y,sigma]:')
print([x,y,sigma])

features_batch, _ = _feature_extractor(image_batch, models[mm],mm)

print('model %d, min/max of features in batch: [%s, %s]'%(mm, torch.min(features_batch), torch.max(features_batch))) 

features_each_prf[batch_inds,:,mm] = torch_utils.get_value(features_batch)


Feature types to exclude from the model:
[]
Extracting features for images [0 - 1]
Clearing steerable pyramid features from memory.
Getting features for pRF [x,y,sigma]:
[-0.31842105263157894, 0.02894736842105261, 0.03999999910593033]
Running steerable pyramid feature extraction...
Images array shape is:
torch.Size([2, 1, 240, 240])
time elapsed = 0.39375
Computing higher order correlations...
time elapsed = 0.23951
Final size of features concatenated is [2 x 641]
Feature types included are:
['pixel_stats', 'mean_magnitudes', 'mean_realparts', 'marginal_stats_lowpass_recons', 'variance_highpass_resid', 'magnitude_feature_autocorrs', 'lowpass_recon_autocorrs', 'highpass_resid_autocorrs', 'magnitude_within_scale_crosscorrs', 'real_within_scale_crosscorrs', 'magnitude_across_scale_crosscorrs', 'real_imag_across_scale_crosscorrs', 'real_spatshift_within_scale_crosscorrs', 'real_spatshift_across_scale_crosscorrs']
model 204, min/max of features in batch: [tensor(-281.7485, device='cuda:0'),

In [9]:
features_batch.dtype

torch.float32

In [10]:
features_each_prf.dtype

dtype('float64')

In [26]:
np.any(torch_utils.get_value(torch.sum(features_batch, axis=0)==0))

True

In [33]:
torch.all(features_batch[:,30:38]==0, axis=0)

tensor([False, False, False, False, False, False, False, False],
       device='cuda:0')

In [36]:
thing = torch.tensor(np.array([[1,2,3],[0,0,0]]))

In [42]:
torch.all(thing, axis=1)

tensor([ True, False])

In [28]:
features_batch[:,30:38]

tensor([[ 3.6532e-06, -9.0402e-07, -1.5367e-07,  1.0408e-06, -1.0979e-08,
         -4.1182e-09, -5.8208e-11,  3.9290e-09],
        [ 2.7426e-06, -9.9302e-07, -1.0435e-06, -4.4724e-06, -1.7801e-08,
         -5.5879e-09,  5.8208e-11,  6.6939e-09]], device='cuda:0')

In [8]:
features_batch[:,30:38]

tensor([[ 3.6532e-06, -9.0402e-07, -1.5367e-07,  1.0408e-06, -1.0979e-08,
         -4.1182e-09, -5.8208e-11,  3.9290e-09],
        [ 2.7426e-06, -9.9302e-07, -1.0435e-06, -4.4724e-06, -1.7801e-08,
         -5.5879e-09,  5.8208e-11,  6.6939e-09]], device='cuda:0')

In [132]:
compute_features = False
feature_types_exclude = []
do_varpart=True
group_all_hl_feats=True
_fmaps_fn = steerable_pyramid_extractor(pyr_height = n_sf, n_ori = n_ori)
# Initialize the "texture" model which builds on first level feature maps
_feature_extractor = texture_feature_extractor(_fmaps_fn,sample_batch_size=batch_size, feature_types_exclude=feature_types_exclude, \
                                               n_prf_sd_out=n_prf_sd_out, aperture=aperture, do_varpart = do_varpart, \
                              compute_features=compute_features, group_all_hl_feats = group_all_hl_feats, device=device)


Feature types to exclude from the model:
[]


In [126]:
np.sum(_feature_extractor.feature_column_labels==3)

0

In [127]:
_feature_extractor.feature_types_include

['pixel_stats',
 'mean_magnitudes',
 'mean_realparts',
 'marginal_stats_lowpass_recons',
 'variance_highpass_resid',
 'magnitude_feature_autocorrs',
 'lowpass_recon_autocorrs',
 'highpass_resid_autocorrs',
 'magnitude_within_scale_crosscorrs',
 'real_within_scale_crosscorrs',
 'magnitude_across_scale_crosscorrs',
 'real_imag_across_scale_crosscorrs',
 'real_spatshift_within_scale_crosscorrs',
 'real_spatshift_across_scale_crosscorrs']

In [128]:
_feature_extractor.init_for_fitting([240,240], models, device)
masks, names = _feature_extractor.get_partial_versions()

Clearing precomputed features from memory.


In [114]:
masks.shape

(3, 641)

In [129]:
masks[0:4,0:50]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.]])

In [89]:
masks[:,0:50]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.]])

In [117]:
np.where(_feature_extractor.feature_column_labels==1)[0][0]

49

In [92]:
np.where(_feature_extractor.feature_column_labels==1)[0][0]

38

In [75]:
_feature_extractor.group_all_hl_feats

True

In [118]:
names

['full_model', 'just_lower-level', 'just_higher-level']

In [77]:
_feature_extractor.

SyntaxError: invalid syntax (<ipython-input-77-41fc8936abb2>, line 1)

In [119]:
image_inds = np.array([50,1,90,15])
mm=1
feats, _ = _feature_extractor(image_inds, models[mm], mm)

Loading pre-computed features from /user_data/mmhender/features/pyramid_texture/S1_features_each_prf_4ori_4sf.h5py
Took 0.71024 seconds to load file
Size of features array for this image set is:
(4, 641, 875)
pixel_stats, size is:
torch.Size([4, 6])
mean_magnitudes, size is:
torch.Size([4, 16])
mean_realparts, size is:
torch.Size([4, 16])
marginal_stats_lowpass_recons, size is:
torch.Size([4, 10])
variance_highpass_resid, size is:
torch.Size([4, 1])
magnitude_feature_autocorrs, size is:
torch.Size([4, 272])
lowpass_recon_autocorrs, size is:
torch.Size([4, 73])
highpass_resid_autocorrs, size is:
torch.Size([4, 25])
magnitude_within_scale_crosscorrs, size is:
torch.Size([4, 24])
real_within_scale_crosscorrs, size is:
torch.Size([4, 24])
magnitude_across_scale_crosscorrs, size is:
torch.Size([4, 48])
real_imag_across_scale_crosscorrs, size is:
torch.Size([4, 96])
real_spatshift_within_scale_crosscorrs, size is:
torch.Size([4, 10])
real_spatshift_across_scale_crosscorrs, size is:
torch.Siz

In [123]:
feats[:, masks[2,:]==1].shape

torch.Size([4, 592])

In [38]:
feats[1,0:10].dtype

torch.float32

In [37]:
features_each_prf[1,0:10,1].dtype

dtype('float64')